### This tutorial presents how to simulate the full network with constant input currents

## Step 1: Import necessary packages and modules

In [ ]:
# This line allows you to display matplotlib plots within the Jupyter Notebook
%matplotlib inline

# Import External packages 

import os
import numpy as np
import matplotlib.pyplot as plt

# Configure the working directory (Important: This should be set to home directory of 'dynworm' folder)

default_dir = os.path.dirname(os.getcwd())
os.chdir(default_dir)

# Import Main module

import dynworm as dw

## Step 2: Initialize neural parameters and connectivity matrices
### Before running any simulation, neural parameters and network connectivity matrices must be initialized 
### This process is carried out by 
### 1) dynworm.network_sim.initialize_params_neural() : initializes neural physiological parameters
### 2) dynworm.network_sim.initialize_connectivity() : initializes connectome data (gap + synaptic)
### Note that neural parameters must be initialized ahead of connectivity matrices
### When not given any arguments (as shown below), the initialization functions can be used to reset the parameter space to default values 

In [ ]:
dw.network_sim.initialize_params_neural()
dw.network_sim.initialize_connectivity()

## Step 3: Define constant input and ablation vectors
### Constant input should be a vector of shape (279, ) where each index defines the constant current (pA unit) for that neuron
### Ablation vector is a boolean vector of shape (279, ) where True = not ablated, False = ablated.

In [ ]:
input_vec = np.zeros(dw.network_sim.params_obj_neural['N'])
ablation_mask = np.ones(dw.network_sim.params_obj_neural['N'], dtype = 'bool')

## Step 4: Configure constant stimuli into neurons
### dynworm.utils.neuron_names_2_inds() function allows you to find the indices of the neurons of your interest

In [ ]:
print(dw.utils.neuron_names_2_inds(['PLML']), dw.utils.neuron_names_2_inds(['PLMR']))

### In this example, we are injecting 2000pA constant current into PLMR (276th index) and PLML (278th index)

In [ ]:
input_vec[276] = 2000 #pA
input_vec[278] = 2000

## Step 5: Run the simulation
### network_sim.run_network_constinput simulates the network when given a constant input. It takes following parameters 
### t_duration: time duration of the simulation in seconds
### input_vec: the input vector with configured constant input currents
### ablation_mask: the ablation vector which define any neurons to be ablated
### In this example, we set t_duration = 15s

In [ ]:
result_dict_network = dw.network_sim.run_network_constinput(t_duration=15, 
                                                    input_vec = input_vec, ablation_mask = ablation_mask,
                                                    use_julia_engine = False)

## Step 6: Extract the result
### run_network_constinput outputs a dictionary object with following keys
### 'v_solution' :   normalized voltage activities of all neurons in respect to their resting potential
### 'steps' : number of solution points between t_start and t_final
### 't' : time vector of the simulation 
### 'raw_v_solution':  un-normalized raw membrane voltage activities of all neurons 
### 'v_threshold': resting potentials of all neurons given the constant input profile 

In [ ]:
result_dict_network.keys()

### In this example, we will take a look at the 'v_solution' 
### We take transpose of 'v_solution' to change its shape to spatial axis of neurons x temporal axis 

In [ ]:
v_sol = result_dict_network['v_solution'].T

### One can use matplotlib.pyplot.pcolor to visualize the activities of all neurons 

In [ ]:
fig = plt.figure(figsize=(15, 8))
plt.pcolor(v_sol[:, 100:], cmap='bwr', vmin = -10, vmax = 10)
plt.ylim(279, 0)
plt.colorbar()

### Or only the motorneuronal activities

In [ ]:
fig = plt.figure(figsize=(15, 8))
plt.pcolor(v_sol[dw.neurons_idx.motor_group, 100:], cmap='bwr', vmin = -3, vmax = 3)
plt.ylim(len(dw.neurons_idx.motor_group), 0)
plt.colorbar()

### Here we show the activities of Ventral Type B, D and Dorsal Type B, D neurons

In [ ]:
fig = plt.figure(figsize=(12, 8))
plt.subplot(4,1,1)
plt.pcolor(v_sol[dw.neurons_idx.VB_ind, 100:600], cmap='bwr')
plt.ylim(len(dw.neurons_idx.VB_ind), 0)
plt.colorbar()

fig = plt.figure(figsize=(12, 8))
plt.subplot(4,1,2)
plt.pcolor(v_sol[dw.neurons_idx.VD_ind, 100:600], cmap='bwr')
plt.ylim(len(dw.neurons_idx.VD_ind), 0)
plt.colorbar()

fig = plt.figure(figsize=(12, 8))
plt.subplot(4,1,3)
plt.pcolor(v_sol[dw.neurons_idx.DB_ind, 100:600], cmap='bwr')
plt.ylim(len(dw.neurons_idx.DB_ind), 0)
plt.colorbar()

fig = plt.figure(figsize=(12, 8))
plt.subplot(4,1,4)
plt.pcolor(v_sol[dw.neurons_idx.DD_ind, 100:600], cmap='bwr')
plt.ylim(len(dw.neurons_idx.DD_ind), 0)
plt.colorbar()